<a href="https://colab.research.google.com/github/Pedrohero13/hadoop/blob/main/tema1_3_analisis_regresion_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

In [ ]:
!java --version

openjdk 11.0.15 2022-04-19
OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1)
OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)


In [ ]:
!echo $JAVA_HOME

/usr/lib/jvm/java-8-openjdk-amd64


In [ ]:
!echo $SPARK_HOME

/content/spark-2.4.8-bin-hadoop2.7


In [ ]:
import findspark
findspark.init()
from google.colab import files
from pyspark import SparkContext, SparkConf,SQLContext,HiveContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql.functions import isnan, when, count, col, lit
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
spark = SparkSession \
    .builder \
    .getOrCreate()

In [ ]:
spark.version

'2.4.8'

In [ ]:
path_csv="/content/BostonHousing.csv"
dataset=spark.read.csv(path_csv,inferSchema=True,header=True)

dataset.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- black: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [ ]:
assembler = VectorAssembler(inputCols=['crim','zn','indus','chas','nox','rm','age','dis','rad','tax','ptratio','black','lstat'],outputCol='Atributes')
output=assembler.transform(dataset)

In [ ]:
finalized_data=output.select("Atributes","medv")
finalized_data.show(10)

+--------------------+----+
|           Atributes|medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03237,0.0,2.18...|33.4|
|[0.06905,0.0,2.18...|36.2|
|[0.02985,0.0,2.18...|28.7|
|[0.08829,12.5,7.8...|22.9|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.17004,12.5,7.8...|18.9|
+--------------------+----+
only showing top 10 rows



In [ ]:
train_data,test_data=finalized_data.randomSplit([0.8,0.2])

In [ ]:
regressor=LinearRegression(featuresCol='Atributes',labelCol='medv')

regressor=regressor.fit(train_data)

print("The coefficient of the model is: %a", regressor.coefficients)
print("The Intercept of the model is: %f", regressor.intercept)

The coefficient of the model is: %a [-0.13396044107045607,0.042143405616880285,0.019212540765338404,2.3213536334196703,-17.851009885362885,3.7130238429373392,-3.349943561358203e-05,-1.5416932461288662,0.33456923717206105,-0.011377065053121416,-0.9999879493341078,0.011508379802387406,-0.552178963514392]
The Intercept of the model is: %f 37.4230425755322


In [ ]:
Pred_lr=regressor.evaluate(test_data)

Pred_lr.predictions.show(10)

+--------------------+----+------------------+
|           Atributes|medv|        prediction|
+--------------------+----+------------------+
|[0.00906,90.0,2.9...|32.2|31.183266728655852|
|[0.01301,35.0,1.5...|32.7| 30.18447174792808|
|[0.01709,90.0,2.0...|30.1| 24.70347315819367|
|[0.02009,95.0,2.6...|50.0| 42.98095196677993|
|[0.03113,0.0,4.39...|17.5| 16.40991120977308|
|[0.03615,80.0,4.9...|27.9|31.599753926764386|
|[0.03738,0.0,5.19...|20.7|21.474279318459665|
|[0.04544,0.0,3.24...|19.8|21.792869254140193|
|[0.0456,0.0,13.89...|23.3|26.443126886080833|
|[0.04666,80.0,1.5...|30.3|  32.3741670176177|
+--------------------+----+------------------+
only showing top 10 rows



In [ ]:
from pyspark.ml.regression import RandomForestRegressor
rfr = RandomForestRegressor(featuresCol="Atributes",labelCol="medv")

In [ ]:
rfr=rfr.fit(train_data)
pred_rfr=rfr.transform(test_data)

pred_rfr.select('Atributes','medv','prediction').show(10)

+--------------------+----+------------------+
|           Atributes|medv|        prediction|
+--------------------+----+------------------+
|[0.00906,90.0,2.9...|32.2| 30.55199685947602|
|[0.01301,35.0,1.5...|32.7|30.586862489161184|
|[0.01709,90.0,2.0...|30.1|28.596951140348533|
|[0.02009,95.0,2.6...|50.0| 46.11686232517482|
|[0.03113,0.0,4.39...|17.5|20.926658193898618|
|[0.03615,80.0,4.9...|27.9|25.939009276790216|
|[0.03738,0.0,5.19...|20.7|23.885501052285814|
|[0.04544,0.0,3.24...|19.8|22.771016502160126|
|[0.0456,0.0,13.89...|23.3|20.211470080069482|
|[0.04666,80.0,1.5...|30.3|32.665093503609526|
+--------------------+----+------------------+
only showing top 10 rows



In [ ]:
from pyspark.ml.regression import GBTRegressor
gbt=GBTRegressor(featuresCol='Atributes',labelCol='medv')

gbt=gbt.fit(train_data)

pred_gbt=gbt.transform(test_data)

pred_gbt.select('Atributes','medv','prediction').show(10)

+--------------------+----+------------------+
|           Atributes|medv|        prediction|
+--------------------+----+------------------+
|[0.00906,90.0,2.9...|32.2| 36.16471997676015|
|[0.01301,35.0,1.5...|32.7|34.596725019940706|
|[0.01709,90.0,2.0...|30.1| 33.64492465750046|
|[0.02009,95.0,2.6...|50.0| 48.43475604783278|
|[0.03113,0.0,4.39...|17.5| 19.79058841037358|
|[0.03615,80.0,4.9...|27.9| 30.55149052708153|
|[0.03738,0.0,5.19...|20.7|23.859847361922814|
|[0.04544,0.0,3.24...|19.8|20.734666969799925|
|[0.0456,0.0,13.89...|23.3|21.645428129127936|
|[0.04666,80.0,1.5...|30.3| 32.65211394722216|
+--------------------+----+------------------+
only showing top 10 rows



In [ ]:
import numpy as np
print("Note: the last rows are the information for intercept")
print("##","--------------------------------")
print("##"," Estimate | Std.Error | t Values | P-value")
coef = np.append(list(regressor.coefficients),regressor.intercept)
Summary=regressor.summary

for i in range(len(Summary.pValues)):
  print("##",'{:10.6f}'.format(coef[i]),\
        '{:10.6f}'.format(Summary.coefficientStandardErrors[i]),\
        '{:8.3f}'.format(Summary.tValues[i]),\
        '{:10.6f}'.format(Summary.pValues[i]))

Note: the last rows are the information for intercept
## --------------------------------
##  Estimate | Std.Error | t Values | P-value
##  -0.133960   0.034591   -3.873   0.000125
##   0.042143   0.015504    2.718   0.006844
##   0.019213   0.068159    0.282   0.778182
##   2.321354   0.980385    2.368   0.018363
## -17.851010   4.358813   -4.095   0.000051
##   3.713024   0.453682    8.184   0.000000
##  -0.000033   0.014692   -0.002   0.998182
##  -1.541693   0.228134   -6.758   0.000000
##   0.334569   0.072317    4.626   0.000005
##  -0.011377   0.004126   -2.757   0.006094
##  -0.999988   0.144930   -6.900   0.000000
##   0.011508   0.003006    3.829   0.000149
##  -0.552179   0.055578   -9.935   0.000000
##  37.423043   5.700987    6.564   0.000000


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
eval_lr=RegressionEvaluator(labelCol="medv",predictionCol="prediction",metricName="rmse")

print("Linear regression model\n")

rmse=eval_lr.evaluate(Pred_lr.predictions)
print("RMSE: %3.f"%rmse)

mse=eval_lr.evaluate(Pred_lr.predictions,{eval_lr.metricName: "mse"})
print("MSE: %.3f"%mse)

mae=eval_lr.evaluate(Pred_lr.predictions,{eval_lr.metricName: "mae"})
print("MAE: %.3f"%mae)

r2=eval_lr.evaluate(Pred_lr.predictions,{eval_lr.metricName:"r2"})
print("r2: %3f"%r2)

Linear regression model

RMSE:   5
MSE: 21.341
MAE: 3.408
r2: 0.733662


In [ ]:
eval_rfr=RegressionEvaluator(labelCol="medv",predictionCol="prediction",metricName="rmse")
print("Regression Forrest model\n")

rmse=eval_rfr.evaluate(pred_rfr)
print("RMSE: %3f"%rmse)

mse=eval_rfr.evaluate(pred_rfr,{eval_rfr.metricName: "mse"})
print("MSE: %3f"%mse)

mae=eval_rfr.evaluate(pred_rfr,{eval_rfr.metricName: "mae"})
print("MAE: %3f"%mae)

r2=eval_rfr.evaluate(pred_rfr,{eval_rfr.metricName: "r2"})
print("r2: %3f"%r2)

Regression Forrest model

RMSE: 3.021995
MSE: 9.132455
MAE: 2.135463
r2: 0.886025


In [ ]:
eval_gbt=RegressionEvaluator(labelCol="medv",predictionCol="prediction",metricName="rmse")
print("Gradient Boot Tree model\n")

rmse=eval_gbt.evaluate(pred_gbt)
print("RMSE: %3f"%rmse)

mse=eval_gbt.evaluate(pred_gbt,{eval_gbt.metricName: "mse"})
print("MSE: %3f"%mse)

mae=eval_gbt.evaluate(pred_gbt,{eval_gbt.metricName: "mae"})
print("MAE: %3f"%mae)

r2=eval_gbt.evaluate(pred_gbt,{eval_gbt.metricName: "r2"})
print("r2: %3f"%r2)

Gradient Boot Tree model

RMSE: 4.394054
MSE: 19.307708
MAE: 2.855992
r2: 0.759035
